In [ ]:
import sys
import matplotlib.pyplot as plt
from glob import glob
import xarray as xr
import rasterio as rio
import rioxarray
import os
import numpy as np

In [ ]:
!{sys.executable} -m pip install phase_o_matic

In [ ]:
from phase_o_matic import presto_phase_delay

In [ ]:
# load in single igram and other data 
def hyp3_to_xarray_single(path):
    '''
    Reads unwrapped phase, coherence, and DEM into xarray dataset from single hyp3 folder 
    '''
    # globs for data to load
    unw_phase_path = glob(f'{path}/*unw_phase.tif')[0]
    dem_path = glob(f'{hyp3_path}/*dem.tif')[0]
    inc_path = glob(f'{hyp3_path}/*_inc_map.tif')
    meta_path = glob(f'{hyp3_path}/*meta.txt')[0]

    # list granules for coordinate
    granule = os.path.split(unw_phase_path)[-1][0:-14]

    # read metadata into dictionary
    d = {}
    with open(meta_path) as f:
        for line in f:
            (key, val) = line.split(':')
            d[key] = str.strip(val)

    # read unw_phase into data array and assign coordinates
    da = xr.open_dataset(unw_phase_path)
    for item in d.keys():
        da = da.assign_coords({'granule':('granule', [granule])})
        da = da.assign_coords({item:('granule', [d[item]])})
    # concatenate into dataset and rename variable
    ds = da.rename({'band_data': 'unw_phase'})

    #open coherence and dem into datasets
    inc_ds = xr.open_dataset(inc_path)
    dem_ds = xr.open_dataset(dem_path)

    # add coherence and dem to unw_phase dataset
    ds['inc_angle'] = (('band', 'y', 'x'), inc_ds.band_data.values)
    ds['elevation'] = (('band', 'y', 'x'), dem_ds.band_data.values)

    # remove band coordinate
    ds = ds.squeeze()

    return ds

In [ ]:
def calc_delay(ds, work_dir):
    """
    """
    time1 = pd.to_datetime(ds.granule[0].item()[5:20], utc=True)
    time2 = pd.to_datetime(ds.granule[0].item()[21:36], utc=True)

    t1 = presto_phase_delay(date=time1, dem=ds.elevation, inc=ds.inc_angle, work_dir=work_dir, wavelength=0.55465763)
    t2 = presto_phase_delay(date=time2, dem=ds.elevation, inc=ds.inc_angle, work_dir=work_dir, wavelength=0.55465763)

    delay_change = t2.isel(time = 0)['delay'] - t1.isel(time = 0)['delay']
    
    ds['era5_phase'] = (('y', 'x'), delay_change.values) 

    return ds

In [ ]:
def interpolate_gaps():
    ds['unw_phase'] = ds.unw_phase.interpolate_na(dim=1, max_gap=10)

In [ ]:
def crop_to_tiles:

In [ ]:
def data_to_hyp3:  